<a href="https://colab.research.google.com/github/xmks-colab/SmartCutter/blob/main/modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from IPython.display import display, HTML
#@title Instruções{ display-mode: "form" }
html_content = """
<div style='background-color: #e7f3ff; padding: 15px; border-radius: 8px; margin: 15px 0; color: #000000; font-family: Arial, sans-serif;'>
  <h3 style='color: #1a73e8; margin-bottom: 10px;'>📋 Informações</h3>

  <h4 style='color: #202124; margin-top: 20px; margin-bottom: 10px;'>Modelo Pré-treinado</h4>
    <li><strong>LIMITE_MERGE_ULTIMO:</strong> Se o último pedaço for menor que x, junta-se com a imagem anterior. Assim, evita pedaços muito pequenos.</li>
    <li><strong>MARGEM_CORTE:</strong> Essa é uma margem de segurança. Quando é encontrada uma área segura para o corte, adiciona a margem de segurança em px para o corte não ser muito próximo do diálogo.</li>
    <li><strong>Os dados são apagados a cada execução!
    </li>
  </ul>
  </p>
</div>
"""

display(HTML(html_content))

In [ ]:

#@title Modelo treinado { display-mode: "form" }
import os
import glob
import numpy as np
import cv2
from PIL import Image
from pathlib import Path
import urllib.request
import shutil
from time import time
from google.colab import files

# ===== CONFIGURAÇÕES =====
ALTURA_CORTE = 4000           #@param {"type":"number"}
THRESHOLD_TEXTO = 0.5      #@param {"type":"number"}
MARGEM_CORTE = 100             #@param {"type":"number"}
# se a imagem final for menor que que x, ele junta com a imagem anterio. evitando pedaços muito pequenos.
LIMITE_MERGE_ULTIMO = 1000    #@param {"type":"number"}
PROCESSAR_TODAS = True
ZIPAR_RESULTADO = True   # True para gerar ZIP no final
PASTA_ORIGEM = "uploads" # Pasta com as imagens originais
PASTA_DESTINO = "cortes" # Pasta onde serão salvos os cortes
FORMATO_SAIDA = "JPG"    #@param ["JPG", "PNG"]
QUALIDADE_JPG = 95       #@param {type:"slider", min:0, max:100}
# =========================



base_dir = "/content"
uploads_dir = os.path.join(base_dir, "uploads")
cortes_dir = os.path.join(base_dir, "cortes")

arquivos_uploads = []
if os.path.exists(uploads_dir):
    arquivos_uploads = [f for f in os.listdir(uploads_dir) if os.path.isfile(os.path.join(uploads_dir, f))]

if os.path.exists(uploads_dir):
    for f in os.listdir(uploads_dir):
        caminho = os.path.join(uploads_dir, f)
        if os.path.isfile(caminho):
            os.remove(caminho)
        elif os.path.isdir(caminho):
            shutil.rmtree(caminho)

if os.path.exists(cortes_dir):
    for f in os.listdir(cortes_dir):
        caminho = os.path.join(cortes_dir, f)
        if os.path.isfile(caminho):
            os.remove(caminho)
        elif os.path.isdir(caminho):
            shutil.rmtree(caminho)

for f in arquivos_uploads:
    caminho = os.path.join(base_dir, f)
    if os.path.isfile(caminho):
        os.remove(caminho)



def resetar_pastas():
    for pasta in [PASTA_ORIGEM, PASTA_DESTINO]:
        if os.path.exists(pasta):
            shutil.rmtree(pasta)
        os.makedirs(pasta)

def fazer_upload_imagens():
    print("📤 Por favor, faça o upload das imagens agora.")
    uploaded = files.upload()
    for nome_arquivo, conteudo in uploaded.items():
        caminho_destino = os.path.join(PASTA_ORIGEM, nome_arquivo)
        with open(caminho_destino, 'wb') as f:
            f.write(conteudo)
    print(f"✅ {len(uploaded)} imagens uploadadas para {PASTA_ORIGEM}.")

def baixar_modelo_east():
    """Baixa o modelo EAST se não existir."""
    modelo_path = "east.pb"
    if not os.path.exists(modelo_path):
        print("📥 Baixando modelo EAST...")
        url = "https://github.com/oyyd/frozen_east_text_detection.pb/raw/master/frozen_east_text_detection.pb"
        urllib.request.urlretrieve(url, modelo_path)
        print("✅ Modelo baixado!")
    return modelo_path

def detectar_texto_regioes(imagem_path, modelo_east_path):
    """Detecta regiões com texto na imagem e retorna lista de (y1, y2)."""
    imagem = cv2.imread(imagem_path)
    if imagem is None:
        return []

    altura_orig, largura_orig = imagem.shape[:2]
    nova_altura = (altura_orig // 32) * 32
    nova_largura = (largura_orig // 32) * 32
    if nova_altura == 0 or nova_largura == 0:
        return []

    imagem_redim = cv2.resize(imagem, (nova_largura, nova_altura))
    blob = cv2.dnn.blobFromImage(imagem_redim, 1.0, (nova_largura, nova_altura),
                                 (123.68, 116.78, 103.94), swapRB=True, crop=False)

    net = cv2.dnn.readNet(modelo_east_path)
    net.setInput(blob)
    (scores, geometry) = net.forward(["feature_fusion/Conv_7/Sigmoid", "feature_fusion/concat_3"])

    regioes_texto = []
    altura_feature, largura_feature = scores.shape[2:4]
    for y in range(altura_feature):
        scores_linha = scores[0, 0, y]
        geo_linha = geometry[0, :, y]
        for x in range(largura_feature):
            if scores_linha[x] < THRESHOLD_TEXTO:
                continue
            h = geo_linha[0, x] + geo_linha[2, x]
            w = geo_linha[1, x] + geo_linha[3, x]
            offset_x, offset_y = x * 4.0, y * 4.0
            x1 = int((offset_x - w / 2) * largura_orig / nova_largura)
            y1 = int((offset_y - h / 2) * altura_orig / nova_altura)
            x2 = int((offset_x + w / 2) * largura_orig / nova_largura)
            y2 = int((offset_y + h / 2) * altura_orig / nova_altura)
            regioes_texto.append((y1, y2))
    return sorted(regioes_texto, key=lambda x: x[0])

def encontrar_melhor_ponto_corte(altura_total, ponto_desejado, regioes_texto):
    ponto = ponto_desejado
    while True:
        conflito = False
        for y1, y2 in regioes_texto:
            if y1 <= ponto <= y2:
                conflito = True
                if y2 + MARGEM_CORTE < altura_total:
                    ponto = y2 + MARGEM_CORTE
                elif y1 - MARGEM_CORTE > 0:
                    ponto = y1 - MARGEM_CORTE
                break
        if not conflito:
            break
    return min(ponto, altura_total)

def obter_info_imagens(imagens):
    info_imagens = []
    y_acumulado = 0
    largura_max = 0
    for caminho in imagens:
        with Image.open(caminho) as img:
            largura, altura = img.size
            info_imagens.append({
                'caminho': caminho,
                'altura': altura,
                'largura': largura,
                'y_inicio': y_acumulado,
                'y_fim': y_acumulado + altura
            })
            y_acumulado += altura
            largura_max = max(largura_max, largura)
    return info_imagens, y_acumulado, largura_max

def mapear_regioes_texto(imagens, modelo_east):
    regioes_texto_global = []
    for info in imagens:
        regioes = detectar_texto_regioes(info['caminho'], modelo_east)
        for y1, y2 in regioes:
            regioes_texto_global.append((y1 + info['y_inicio'], y2 + info['y_inicio']))
    return sorted(regioes_texto_global, key=lambda x: x[0])

def calcular_pontos_corte(altura_total, regioes_texto):
    """Calcula pontos de corte no espaço global."""
    pontos_corte = []
    y_atual = 0
    while y_atual < altura_total:
        proximo_y = min(y_atual + ALTURA_CORTE, altura_total)
        if regioes_texto and proximo_y < altura_total:
            regioes_relevantes = [(y1, y2) for y1, y2 in regioes_texto
                                  if abs(y1 - proximo_y) <= 200 or abs(y2 - proximo_y) <= 200]
            if regioes_relevantes:
                proximo_y = encontrar_melhor_ponto_corte(altura_total, proximo_y, regioes_relevantes)
        pontos_corte.append((y_atual, proximo_y))
        y_atual = proximo_y
    return pontos_corte

def cortar_imagens_globais(info_imagens, pontos_corte, largura_max, pasta_destino):
    """Corta as imagens com base nos pontos de corte globais."""
    num_pedaco = 1
    ext = "png" if FORMATO_SAIDA == "PNG" else "jpg"
    for y_inicio_global, y_fim_global in pontos_corte:
        # Cria uma nova imagem para o pedaço
        altura_pedaco = y_fim_global - y_inicio_global
        pedaco = Image.new('RGB', (largura_max, altura_pedaco), (255, 255, 255))

        y_offset = 0
        for info in info_imagens:
            y_inicio_img = info['y_inicio']
            y_fim_img = info['y_fim']
            caminho = info['caminho']
            largura_img = info['largura']

            if y_inicio_global < y_fim_img and y_fim_global > y_inicio_img:
                with Image.open(caminho) as img:

                    y_inicio_local = max(0, y_inicio_global - y_inicio_img)
                    y_fim_local = min(info['altura'], y_fim_global - y_inicio_img)


                    area_corte = (0, y_inicio_local, largura_img, y_fim_local)
                    parte_img = img.crop(area_corte)


                    pedaco.paste(parte_img, (0, y_offset))
                    y_offset += (y_fim_local - y_inicio_local)


        nome_pedaco = f"{num_pedaco:02d}.{ext}"
        caminho_salvar = os.path.join(pasta_destino, nome_pedaco)
        if FORMATO_SAIDA == "JPG":
            pedaco.save(caminho_salvar, quality=QUALIDADE_JPG)
        else:
            pedaco.save(caminho_salvar)
        print(f"✅ Pedaço {nome_pedaco} ({largura_max}x{altura_pedaco}px) salvo")
        num_pedaco += 1
    return num_pedaco

def main():

    resetar_pastas()


    fazer_upload_imagens()

    modelo_east = baixar_modelo_east()


    imagens = []
    for ext in ['*.jpg', '*.jpeg', '*.png', '*.webp']:
        imagens.extend(glob.glob(os.path.join(PASTA_ORIGEM, ext)))
    imagens.sort()

    if not imagens:
        print("❌ Nenhuma imagem encontrada!")
        return

    print(f"📸 {len(imagens)} imagens encontradas.")
    inicio = time()


    info_imagens, altura_total, largura_max = obter_info_imagens(imagens)
    print(f"📏 Altura total do painel virtual: {altura_total}px")
    print(f"📏 Largura máxima: {largura_max}px")


    print("🔍 Detectando texto em todas as imagens...")
    regioes_texto = mapear_regioes_texto(info_imagens, modelo_east)
    print(f"📝 {len(regioes_texto)} região(ões) de texto detectada(s)")


    print("🎯 Calculando pontos de corte...")
    pontos_corte = calcular_pontos_corte(altura_total, regioes_texto)


    if LIMITE_MERGE_ULTIMO > 0 and len(pontos_corte) > 1:
        last_height = pontos_corte[-1][1] - pontos_corte[-1][0]
        if last_height < LIMITE_MERGE_ULTIMO:
            pontos_corte[-2] = (pontos_corte[-2][0], pontos_corte[-1][1])
            del pontos_corte[-1]
            print("🔄 Último pedaço pequeno mesclado com o anterior.")

    print(f"✂️ {len(pontos_corte)} pedaços serão gerados")


    print("🔪 Cortando imagens...")
    num_pedacos = cortar_imagens_globais(info_imagens, pontos_corte, largura_max, PASTA_DESTINO)

    duracao = time() - inicio
    print(f"✅ Corte concluído em {duracao:.2f}s")
    print(f"🎉 Total de {num_pedacos-1} pedaços gerados")

    if ZIPAR_RESULTADO:
        shutil.make_archive("cortes", 'zip', PASTA_DESTINO)
        print("📦 Arquivo cortes.zip criado!")
        zip_path = '/content/cortes.zip'
        files.download(zip_path)

if __name__ == "__main__":
    main()